In [1]:
# from google.colab import drive
# drive.mount('/content/pinn-main')

In [2]:
# cd /content/pinn-main/MyDrive/pinns-main/modulo

In [3]:
import argparse
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import scipy.io
import matplotlib.pyplot as plt

In [4]:
# !pip install wandb -qU
# !pip install ml_collections
import modulus
import data_generator as dg
import default
import wandb

In [5]:
torch.manual_seed(44)
np.random.seed(44)
torch.cuda.manual_seed(44)

      utils.py

In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt


def ac_equation(u, tx):
    u_tx = torch.autograd.grad(u, tx, torch.ones_like(u), create_graph= True)[0]
    u_t = u_tx[:, 0:1]
    u_x = u_tx[:, 1:2]
    u_xx = torch.autograd.grad(u_x, tx, torch.ones_like(u_x), create_graph= True)[0][:, 1:2]
    e = u_t -0.0001*u_xx + 5*u**3 - 5*u
    return e

def resplot(x, t, t_data, x_data, Exact, u_pred):
    plt.figure(figsize=(10, 10))
    plt.subplot(2, 2, 1)
    plt.plot(x, Exact[:,0],'-')
    plt.plot(x, u_pred[:,0],'--')
    plt.legend(['Reference', 'Prediction'])
    plt.title("Initial condition ($t=0$)")

    plt.subplot(2, 2, 2)
    t_step = int(0.25*len(t))
    plt.plot(x, Exact[:,t_step],'-')
    plt.plot(x, u_pred[:,t_step],'--')
    plt.legend(['Reference', 'Prediction'])
    plt.title("$t=0.25$")

    plt.subplot(2, 2, 3)
    t_step = int(0.5*len(t))
    plt.plot(x, Exact[:,t_step],'-')
    plt.plot(x, u_pred[:,t_step],'--')
    plt.legend(['Reference', 'Prediction'])
    plt.title("$t=0.5$")

    plt.subplot(2, 2, 4)
    t_step = int(0.99*len(t))
    plt.plot(x, Exact[:,t_step],'-')
    plt.plot(x, u_pred[:,t_step],'--')
    plt.legend(['Reference', 'Prediction'])
    plt.title("$t=0.99$")
    plt.show()
    plt.close()


In [7]:

class TrainClass:
    def __init__(self, cfg, wandbFlag=False):
        # Конфигурация
        self.num_t = cfg.num_t
        self.num_x = cfg.num_x
        self.num_epochs = cfg.epochs
        self.num_hidden_layers = 4
        self.num_nodes = cfg.hidden_count
        self.learning_rate = cfg.lr
        self.data_path = cfg.data_path
        self.wandbFlag = wandbFlag

        #Подключение отслеживания с помощью wandb
        if self.wandbFlag:
          self.__wandbConnect(cfg)

        # Устройство
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print("Operation mode: ", self.device)

        # Данные
        self.__createData()
        # Модель
        self.model = modulus.pinn(cfg).to(self.device)

        # Оптимизатор
        self.optimizer = torch.optim.Adam(self.model.parameters(), betas=(0.999, 0.999), lr=self.learning_rate)

        # Логирование
        self.loss_history = []
        self.l2_history = []
        self.best_loss = float('inf')
        self.best_epoch = 0

    '''
    Подключается к wandb для отслеживания процесса обучения
    '''
    def __wandbConnect(self, cfg):
        wandb.login()
        wandb.init(
            project=cfg.project,
            name=cfg.name,
            config={
            "epochs": cfg.epochs,
            })
    '''
    Генерирует данные:
      variables - выборка, содержищие граничные и начальные условия
      variables_f - вся выборка
      u_data - содержит решение для выборки variables
    '''
    def __createData(self):
        self.t_data, self.x_data, self.u_data, self.t_data_f, self.x_data_f = dg.ac_generator(self.num_t, self.num_x)
        self.variables = torch.FloatTensor(np.concatenate((self.t_data, self.x_data), axis=1)).to(self.device)
        self.variables_f = torch.FloatTensor(np.concatenate((self.t_data_f, self.x_data_f), axis=1)).to(self.device)
        self.variables_f.requires_grad = True
        self.u_data = torch.FloatTensor(self.u_data).to(self.device)

    '''
    Считает L2 потерю относительно верного решения, находящимся по пути cfg.data_path
    '''
    def __calculate_l2_error(self):
        t = np.linspace(0, 1, 201).reshape(-1, 1)
        x = np.linspace(-1, 1, 513)[:-1].reshape(-1, 1)
        T = t.shape[0]
        N = x.shape[0]
        T_star = np.tile(t, (1, N)).T
        X_star = np.tile(x, (1, T))
        t_test = T_star.flatten()[:, None]
        x_test = X_star.flatten()[:, None]

        test_variables = torch.FloatTensor(np.concatenate((t_test, x_test), axis=1)).to(self.device)
        with torch.no_grad():
            u_pred = self.model(test_variables)
        u_pred = u_pred.cpu().numpy().reshape(N, T)

        # Сравнение с эталоном
        data = scipy.io.loadmat(self.data_path)
        exact_solution = np.real(data['uu'])
        error = np.linalg.norm(u_pred - exact_solution, 2) / np.linalg.norm(exact_solution, 2)
        return error


    
    '''
    Выводит график функции потерь, а также эпоху с наименьшей величиной потерь
    '''
    def printLossGraph(self):
        print(f"[Best][Epoch: {self.best_epoch}] Train loss: {self.best_loss}")
        plt.figure(figsize=(10, 5))
        plt.plot(self.loss_history)
        plt.show()
        
        plt.figure(figsize=(10, 5))
        plt.plot(self.l2_history)
        plt.show()

    '''
    Выводит график вычисленного уравнения
    '''
    def printEval(self):
      #Загружаем лучшие веса
      self.model.load_state_dict(torch.load('./ac_1d.pth'))

      #Подготовка данных и вывод L2
      t = np.linspace(0, 1, 201).reshape(-1,1) # T x 1
      x = np.linspace(-1, 1, 513)[:-1].reshape(-1,1) # N x 1
      T = t.shape[0]
      N = x.shape[0]
      T_star = np.tile(t, (1, N)).T  # N x T
      X_star = np.tile(x, (1, T))  # N x T
      t_test = T_star.flatten()[:, None]
      x_test = X_star.flatten()[:, None]

      test_variables = torch.FloatTensor(np.concatenate((t_test, x_test), 1)).to(self.device)
      with torch.no_grad():
          u_pred = self.model(test_variables)
      u_pred = u_pred.cpu().numpy().reshape(N,T)

      data = scipy.io.loadmat(self.data_path)
      Exact = np.real(data['uu'])
      err = u_pred-Exact

      err = np.linalg.norm(err,2)/np.linalg.norm(Exact,2)
      print(f"L2 Relative Error: {err}")

      #Рисуем графики
      resplot(x, t, self.t_data, self.x_data, Exact, u_pred)

      plt.figure(figsize=(10, 5))
      plt.imshow(u_pred, interpolation='nearest', cmap='jet',
                  extent=[t.min(), t.max(), x.min(), x.max()],
                  origin='lower', aspect='auto')
      plt.clim(-1, 1)
      plt.ylim(-1,1)
      plt.xlim(0,1)
      plt.scatter(self.t_data, self.x_data)
      plt.xlabel('t')
      plt.ylabel('x')
      plt.title('u(t,x)')
      plt.show()

    '''
    Функция тренировки нейросети
    '''
    def train(self):
        for epoch in tqdm(range(self.num_epochs)):
            self.optimizer.zero_grad()

            # Предсказания
            u_pred = self.model(self.variables)
            print(u_pred)
            exit()
            u_pred_f = self.model(self.variables_f)

            # Вычисление функции потерь
            loss_f = torch.mean(ac_equation(u_pred_f, self.variables_f) ** 2)
            loss_u = torch.mean((u_pred - self.u_data) ** 2)
            loss = loss_f + loss_u

            # Обновление весов
            loss.backward()
            self.optimizer.step()

            current_loss = loss.item()
            self.loss_history.append(current_loss)
            l2_error = self.__calculate_l2_error()
            self.l2_history.append(l2_error)

            # Сохранение лучшей модели
            if current_loss < self.best_loss:
                self.best_loss = current_loss
                self.best_epoch = epoch
                torch.save(self.model.state_dict(), f'./ac_1d.pth')

            # Логирование
            if epoch:
                print(f"Epoch {epoch}, Train loss: {current_loss}, L2: {l2_error}")
                
                if self.wandbFlag:
                  wandb.log({"epoche": epoch, "loss": current_loss})
                  wandb.log({"epoche": epoch, "L2": l2_error})

            # if epoch % 500 == 0:
            #     l2_error = self.__calculate_l2_error()
            #     if self.wandbFlag:
            #       wandb.log({"epoche": epoch, "L2": l2_error})
            #     print(f"L2 Relative Error: {l2_error}")
        self.printLossGraph()


In [ ]:
a = TrainClass(default.get_config())
a.train()
# a.printLossGraph()
a.printEval()
# a.train()

<class 'ml_collections.config_dict.config_dict.ConfigDict'>
Operation mode:  cpu


/home/maksim/Calculation-module-based-on-PINN-neural-network-architecture/.venv/lib64/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
  0%|          | 0/11 [00:00<?, ?it/s]

tensor([[-0.1308],
        [ 0.0861],
        [ 0.0802],
        [-0.1618],
        [ 0.1059],
        [-0.1777],
        [-0.1738],
        [ 0.1059],
        [ 0.1130],
        [ 0.0934],
        [-0.1593],
        [ 0.0940],
        [ 0.0676],
        [ 0.0654],
        [-0.1663],
        [ 0.1202],
        [ 0.1263],
        [-0.1606],
        [-0.1103],
        [-0.1247],
        [ 0.1130],
        [ 0.1243],
        [ 0.0855],
        [-0.1769],
        [-0.1735],
        [-0.1777],
        [-0.1697],
        [ 0.0855],
        [-0.1411],
        [ 0.0964],
        [-0.1688],
        [ 0.0710],
        [-0.1688],
        [-0.1697],
        [-0.1746],
        [ 0.1117],
        [ 0.0977],
        [ 0.1156],
        [-0.1688],
        [ 0.0767],
        [-0.1560],
        [-0.1288],
        [-0.1766],
        [-0.1393],
        [-0.1103],
        [-0.1641],
        [ 0.1002],
        [ 0.1065],
        [ 0.0802],
        [-0.1657],
        [-0.1599],
        [-0.1553],
        [-0.

  9%|▉         | 1/11 [00:02<00:28,  2.83s/it]

tensor([[ 0.2439],
        [-0.1370],
        [-0.1264],
        [ 0.2554],
        [-0.1658],
        [ 0.2629],
        [ 0.2609],
        [-0.1658],
        [-0.1735],
        [-0.1488],
        [ 0.2544],
        [-0.1497],
        [-0.1011],
        [-0.0963],
        [ 0.2574],
        [-0.1800],
        [-0.1844],
        [ 0.2549],
        [ 0.2371],
        [ 0.2418],
        [-0.1735],
        [-0.1830],
        [-0.1360],
        [ 0.2625],
        [ 0.2607],
        [ 0.2629],
        [ 0.2589],
        [-0.1360],
        [ 0.2475],
        [-0.1534],
        [ 0.2585],
        [-0.1082],
        [ 0.2585],
        [ 0.2589],
        [ 0.2613],
        [-0.1722],
        [-0.1551],
        [-0.1760],
        [ 0.2585],
        [-0.1197],
        [ 0.2531],
        [ 0.2432],
        [ 0.2623],
        [ 0.2469],
        [ 0.2371],
        [ 0.2564],
        [-0.1586],
        [-0.1665],
        [-0.1264],
        [ 0.2571],
        [ 0.2547],
        [ 0.2528],
        [ 0.

...

 45%|████▌     | 5/11 [00:11<00:12,  2.16s/it]

Epoch 4, Train loss: 0.6094091534614563, L2: 1.0328487933471815
tensor([[-4.7225e-02],
        [ 1.6946e-01],
        [ 1.7150e-01],
        [-4.1494e-02],
        [ 1.6203e-01],
        [-2.8175e-02],
        [-3.3125e-02],
        [ 1.6203e-01],
        [ 1.5928e-01],
        [ 1.6681e-01],
        [-4.2531e-02],
        [ 1.6658e-01],
        [ 1.7534e-01],
        [ 1.7592e-01],
        [-3.9151e-02],
        [ 1.5650e-01],
        [ 1.5424e-01],
        [-4.2024e-02],
        [-4.6203e-02],
        [-4.7179e-02],
        [ 1.5928e-01],
        [ 1.5499e-01],
        [ 1.6967e-01],
        [-2.9458e-02],
        [-3.3514e-02],
        [-2.8175e-02],
        [-3.6830e-02],
        [ 1.6967e-01],
        [-4.6645e-02],
        [ 1.6565e-01],
        [-3.7517e-02],
        [ 1.7439e-01],
        [-3.7517e-02],
        [-3.6830e-02],
        [-3.2335e-02],
        [ 1.5978e-01],
        [ 1.6518e-01],
        [ 1.5827e-01],
        [-3.7517e-02],
        [ 1.7264e-01],
        [-4.3693

 45%|████▌     | 5/11 [00:13<00:15,  2.62s/it]


KeyboardInterrupt: 

: 